# Agents 101

## What is an agent?

Large Language Models (LLMs) trained to perform **causal language modeling** can tackle a wid range of tasks, but they often struggle with basic tasks like logic, calculation, and search.

An **agent** is a system that uses an LLM as its engine, and it has access to functions called *tools*.

These **tools** are functions for performing a task, and they contain all necessary description for the agent to properly use them.

The agent can be programmed to
* devise a series of actions/tools and run them all at once
* plan and execute actions/tools one by one and wait for the outcome of each action before launching the next one.

## How can I build an agent?

To initialize an agent, we need
* an LLM to power our agent - the agent is not exactly the LLM, it's more like the agent is a program that uses an LLM as its engine
* a system prompt - what the LLM engine will be prompted with to generate its output
* a toolbox from which the agent pick tools to execute
* a parser to extract from the LLM output which tools are to call and with which arguments

To start with, we need to install the `agents`

In [ ]:
!pip install transformers[agents] sentence-transformers

Build our LLM engine by defining a `llm_engine` method.

In [ ]:
from huggingface_hub import login, InferenceClient

login('<HUGGINGFACE_HUB_API_TOKEN>')

client = InferenceClient(model='meta-llama/Meta-Llama-3-70B-Instruct')

def llm_engine(messages, stop_sequences=['Task']):
    response = client.chat_completion(
        messages,
        stop=stop_sequences,
        max_tokens=1000,
    )
    answer = response.choices[0].message.content

    return answer

We could use any `llm_engine` method as long as
1. it follows the messages format(`List[Dict[str, str]]`) for its input `messages`, and it returns a `str`.
2. it stops generating outputs at the sequences passed in the argument `stop_sequences`



We will also need a `tools` argument which accepts a list of `Tools` - it can be an empty list. We can also add the default toolbox on top of our `tools` list by defining the optional argument `add_base_tool=True`.

Now we can create an agent and run it. We can also create a `TransformersEngine` with a pre-initialized pipeline to run inference on our local machine using `transformers`. For convenience, since agentic behaviors generally require stronger models such as `Llama-3.1-70B-Instruct` that are difficult to run locally, we can use the `HfApiEngine` class that initializes a `huggingface_hub.InferenceClient` under the hood:

In [ ]:
from transformers import CodeAgent, HfApiEngine

llm_engine = HfApiEngine(model='meta-llama/Meta-Llama-3-70B-Instruct')
agent = CodeAgent(tools=[],
                  llm_engine=llm_engine,
                  add_base_tool=True)

agent.run(
    'Could you translate this sentence from French, say it out loud and return the audio.',
    sentence="Où est la boulangerie la plus proche?",
)

Here we used an additional `sentence` argument. We can pass text as additional arguments to the model.

We can also use this to indicate the path to local or remote files for the model to use:

In [ ]:
from transformers import ReactCodeAgent

agent = ReactCodeAgent(tools=[],
                       llm_engine=llm_engine,
                       add_base_tool=True)

agent.run(
    "Why does Mike not know many people in New York?",
    audio="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3"
)

The prompt and output parser were automatically defined, but we can easily inspect them:

In [ ]:
agent.system_prompt_template

It is important to explain as clearly as possible the task we want to perform. Every `run()` operation is independent, and since an agent is powered by an LLM, minor variations in our prompt might yield completely different results.

We can run an agent consecutively for different tasks: each time the attributes `agent.task` and `agent.logs` will be re-initialized.

### Code execution

A Python interpreter executes the code on a set of inputs passed along with your tools. The Python interpreter also doesn't allow imports by default outside of a safe list, so all the most obvious attacks shouldn't be an issue.

In [ ]:
from transformers import ReactCodeAgent

agent = ReactCodeAgent(tools=[],
                       additional_authorized_imports=['requests', 'bs4'])

agent.run(
    "Could you get me the title of the page at url 'https://huggingface.co'?"
)

## The system prompt

An agent, or rather the LLM that drives the agent, generates an output based on the system prompt. The system prompt can be customized and tailored to the intended task.

For example, check the system prompt for the ReactCodeAgent:
```
You will be given a task to solve as best you can.
You have access to the following tools:
<<tool_descriptions>>

To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task, then the tools that you want to use.
Then in the 'Code:' sequence, you shold write the code in simple Python. The code sequence must end with '/End code' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then be available in the 'Observation:' field, for using this information as input for the next step.

In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using notional tools:
---
{examples}

Above example were using notional tools that might not exist for you. You only have acces to those tools:
<<tool_names>>
You also can perform computations in the python code you generate.

Always provide a 'Thought:' and a 'Code:\n```py' sequence ending with '```<end_code>' sequence. You MUST provide at least the 'Code:' sequence to move forward.

Remember to not perform too many operations in a single code block! You should split the task into intermediate code blocks.
Print results at the end of each step to save the intermediate results. Then use final_answer() to return the final result.

Remember to make sure that variables you use are all defined.

Now Begin!
```

The system prompt includes:
* an *introduction* that explains how the agent should behave and what tools are.
* a description of all the tools that is defined by a `<<tool_description>>` token that is dynamically replaced at runtime with the tools defined/chosen by the user.
  * The tool description comes from the tool attributes, `name`, `description`, `inputs` and `output_type`, and a simple `jinja2` template that we can define.
* The expected output format.

We could improve the system prompt, for example, by adding an explanation of the output format.

For maximum flexibility, we can overwrite the whole system prompt template by passing our custom prompt as an argument to the `system_prompt` parameter

In [ ]:
from transformers import ReactJsonAgent
from transformers.agents import PythonInterpreterTool

agent = ReactJsonAgent(tools=[PythonInterpreterTool()],
                       system_prompt="{our_custom_prompt_here}")

Make sure to define the `<<tool_descriptions>>` string somewhere in the `template` so the agent is aware of the available tools.

## Inspecting an agent run

* `agent.logs` stores the fine-grained logs of the agent.
* Running `agent.write_inner_memory_from_logs()` creates an inner memory of the agent's logs for the LLM to view, as a list of chat messages. This method goes over each step of the log and only stores what it is interested in as a message: for example, it will save the system prompt and task in separate messages, then for each step it will store the LLM output as a message, and the tool call output as another message. Use this if we want a higher-level view of what has happened - but not every log will be transcripted by this method.

## Tools

A tool is an atomic function to be used by an agent.

### Default toolbox

Add a default toolbox to our agent upon initialization with `add_base_tools = True`:
* **Document question answering**: given a document (such as a PDF) in image format, answer a question on this document
* **Image question answering**: given a image, answer a question on this image
* **Speech to text**: gien an audio recording of a person talking, transcribe the speech into text
* **Text to speech**: convert text to speech
* **Translation**: translate a given sentence from source language to target language
* **DuckDuckGo search**: perform a web search using DuckDuckGo brower
* **Python code interpreter**: run the LLM generated Python code in a secure environment.

In [ ]:
from transformers import load_tool

tool = load_tool('text-to-speech')
audio = tool("This is a text to speech tool")

### Create a new tool

Let's create a tool that returns the most downloaded model for a given task from the hub.

In [4]:
from huggingface_hub import list_models

task = 'text-classification'

model = next(iter(list_models(filter=task,
                              sort='downloads',
                              direction=-1)))

model.id

'1231czx/llama3_it_ultra_list_and_bold500'

This code can quickly be converted into a tool, just by wrapping it in a function and adding the `tool` decorator:

In [ ]:
from transformers import tool

@tool
def model_download_tool(task:str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which
    """
    model = next(iter(list_models(filter=task,
                              sort='downloads',
                              direction=-1)))
    return model.id

The function needs:
* A clear name to describe what the tool does.
* Type hints on both inputs and output
* A description that includes an "Args:" part where each argument is described. All these will be automatically baked into the agent's system prompt upon initialization.

Then we can directly initialize our agent:

In [ ]:
from transformers import CodeAgent

agent = CodeAgent(tools=[model_download_tool],
                  llm_engine=llm_engine)

agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)

### Manage agent's toolbox

If we have already initialized an agent, it is inconvenient to reinitialize it from scratch with a tool we want to use.

We can add the `model_download_tool` to an existing agent initialized with only the default toolbox.

In [ ]:
from transformers import CodeAgent

agent = CodeAgent(tools=[],
                  llm_engine=llm_engine,
                  add_base_tool=True)

# add existing tool
agent.toolbox.add_tool(model_download_tool)

agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)

Use the `agent.toolbox.update_tool()` method to replace an existing tool in the agent's toolbox.

### Use a collection of tools

We can leverage tool collections by using the `ToolCollection` object, with the slug of the collection we want to use.

In [ ]:
from transformers import ToolCollection, ReactCodeAgent

image_tool_collection = ToolCollection(collection_slug="huggingface-tools/diffusion-tools-6630bb19a942c2306a2cdb6f")

agent = ReactCodeAgent(tools=[*image_tool_collection],
                       add_base_tools=True) # use default llm_engine

agent.run("Please draw me a picture of rivers and lakes.")